# Pipeline to make dataset from Hypothesis.is

### Install the libraries of need

In [1]:
#!pip3 install elasticsearch
#!pip install pandas requests spacy seaborn

In [2]:
import pandas as pd
import os, json
import requests
import seaborn as sns
import spacy
import hashlib
import matplotlib.pyplot as plt
import numpy as np

### [Optional] Fetch the data (run this only if no data is available in the folder or changes in Hypothes.is are expected)

In [3]:
# Let's see the sample data first.
data = {
    'user': "Ezloplop",
    'group_text': "BehSci",
    'group': "Jk8bYJdN",
    'api_key': "my_api_key"
}

url_search = "https://api.hypothes.is/api/search"
url_ = "https://hypothes.is/groups/Jk8bYJdN/behsci"

In [4]:
res = requests.get(url=url_search, params={'group':'Jk8bYJdN', 'limit': 200}, headers={'Authorization': f"Bearer {data['api_key']}"})

In [5]:
json = res.json()
pd.DataFrame(json['rows']).head(5)

,id,created,updated,user,uri,text,tags,group,permissions,target,document,links,user_info,flagged,hidden
0,EFWynpMTEeyO3G9alGUJkA,2022-02-21T12:37:42.860894+00:00,2022-08-29T10:32:00.921505+00:00,acct:jackiekrauss@hypothes.is,https://securingdemocracy.gmfus.org/rt-deutsch...,"Schafer, B. (2021, October 5). RT Deutsch Find...","[is:blog, lang:en, RT Deutsch, anti-vaccinatio...",Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://securingdemocracy.gmfus.o...,{'title': ['RT Deutsch Finds a Home with Anti-...,{'html': 'https://hypothes.is/a/EFWynpMTEeyO3G...,{'display_name': None},False,False
1,dR7DsExZEeyc5Se_QXZPbw,2021-11-23T12:32:44.190592+00:00,2022-08-29T10:31:48.676007+00:00,acct:jackiekrauss@hypothes.is,https://www.newscientist.com/article/2298169-w...,"Sparkes, M. (2021, November 19). Wikipedia tes...","[is:news, lang:en, wikipedia, Artificial intel...",Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.newscientist.com/arti...,{'title': ['Wikipedia tests AI for spotting co...,{'html': 'https://hypothes.is/a/dR7DsExZEeyc5S...,{'display_name': None},False,False
2,1h_c6CztEeyMa1dlOOluOw,2021-10-14T12:54:15.213834+00:00,2022-08-29T10:31:42.805199+00:00,acct:jackiekrauss@hypothes.is,https://www.theguardian.com/world/2021/oct/11/...,"Henley, J. (2021, October 11). UK’s high Covid...","[is:news, lang:en, COVID-19, United Kingdom, E...",Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.theguardian.com/world...,{'title': ['UK’s high Covid case rates buck tr...,{'html': 'https://hypothes.is/a/1h_c6CztEeyMa1...,{'display_name': None},False,False
3,LVkBykCTEeypZte95Ybu5Q,2021-11-08T12:55:40.654202+00:00,2022-08-29T10:31:42.592685+00:00,acct:jackiekrauss@hypothes.is,https://twitter.com/BenPBradshaw/status/145545...,"Ben Bradshaw. (2021, November 2). More than 50...","[is:tweet, lang:en, COVID-19, parliament, Unit...",Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://twitter.com/BenPBradshaw/...,{'title': ['Ben Bradshaw on Twitter']},{'html': 'https://hypothes.is/a/LVkBykCTEeypZt...,{'display_name': None},False,False
4,TV8KuEuTEey7rLN9r-8Mdw,2021-11-22T12:54:17.160769+00:00,2022-08-29T10:31:42.583167+00:00,acct:jackiekrauss@hypothes.is,https://twitter.com/BlakesWort/status/14587947...,"Blake. (2021, November 11). Wie ungeheuer pein...","[is:tweet, lang:de, COVID-19, vaccine mandate,...",Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://twitter.com/BlakesWort/st...,{'title': ['Blake on Twitter']},{'html': 'https://hypothes.is/a/TV8KuEuTEey7rL...,{'display_name': None},False,False


### Hypothesis.is stores the data by user, so we'll use 'users' info to get data.

In [8]:
users = requests.get(url=f"https://api.hypothes.is/api/groups/{data['group']}/members")

In [9]:
users.json()[1]

{'authority': 'hypothes.is',
 'userid': 'acct:stefanherzog@hypothes.is',
 'username': 'stefanherzog',
 'display_name': 'Stefan Herzog'}

In [10]:
data_per_user = []
for user in users.json():
    print(user)
    user_id = user['userid']
    user_batch = []
    for i in range(0, 5000, 200):
        res = requests.get(url=url_search,
                           params={'group':'Jk8bYJdN', 'user':f'{user_id}', 'limit': 200, 'offset':i},
                           headers={'Authorization': f"Bearer {data['api_key']}"})
        user_batch.append(res.json())
    data_per_user.append(user_batch)

{'authority': 'hypothes.is', 'userid': 'acct:amyhcurtis@hypothes.is', 'username': 'amyhcurtis', 'display_name': None}
{'authority': 'hypothes.is', 'userid': 'acct:stefanherzog@hypothes.is', 'username': 'stefanherzog', 'display_name': 'Stefan Herzog'}
{'authority': 'hypothes.is', 'userid': 'acct:Hahn@hypothes.is', 'username': 'Hahn', 'display_name': None}
{'authority': 'hypothes.is', 'userid': 'acct:Marlene_Wulf@hypothes.is', 'username': 'Marlene_Wulf', 'display_name': None}
{'authority': 'hypothes.is', 'userid': 'acct:edampf@hypothes.is', 'username': 'edampf', 'display_name': None}
{'authority': 'hypothes.is', 'userid': 'acct:lewan@hypothes.is', 'username': 'lewan', 'display_name': None}
{'authority': 'hypothes.is', 'userid': 'acct:gailelhalaby@hypothes.is', 'username': 'gailelhalaby', 'display_name': None}
{'authority': 'hypothes.is', 'userid': 'acct:LeaGlaubig@hypothes.is', 'username': 'LeaGlaubig', 'display_name': None}
{'authority': 'hypothes.is', 'userid': 'acct:Danaeioak@hypothes

In [11]:
total_anns = []
for batch in data_per_user:
    for elem in batch:
        total_anns += elem['rows']

In [13]:
# Store the dataframe into json file
total_anns_df = pd.DataFrame(total_anns)
total_anns_df.to_json('hypothesis_v1__12-03-22.jsonl', orient='records', lines=True)

## 1. Load Data from File

In [14]:
# dataframe of our dataset
df = pd.read_json('hypothesis_v1__12-03-22.jsonl', orient='records', lines=True)

In [15]:
df

,id,created,updated,user,uri,text,tags,group,permissions,target,document,links,user_info,flagged,hidden
0,rnlcIho0EeuKgFfKPBbajQ,2020-10-29T22:18:33.169969+00:00,2020-10-29T22:18:33.169969+00:00,acct:amyhcurtis@hypothes.is,https://www.youtube.com/watch?v=QapwrR9C3Z4,https://twitter.com/i/web/status/1306171100544...,[has:context],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.youtube.com/watch?v=Q...,{'title': ['The impact of Covid-19 on media – ...,{'html': 'https://hypothes.is/a/rnlcIho0EeuKgF...,{'display_name': None},False,False
1,qHdm3ho0EeuTJufnuQnIqQ,2020-10-29T22:18:23.010549+00:00,2020-10-29T22:18:23.010549+00:00,acct:amyhcurtis@hypothes.is,https://www.youtube.com/watch?v=QapwrR9C3Z4,2020-09-16,[has:date],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.youtube.com/watch?v=Q...,{'title': ['The impact of Covid-19 on media – ...,{'html': 'https://hypothes.is/a/qHdm3ho0EeuTJu...,{'display_name': None},False,False
2,oUZLeBo0EeuthtuX8fq6yQ,2020-10-29T22:18:11.003006+00:00,2020-10-29T22:18:11.003006+00:00,acct:amyhcurtis@hypothes.is,https://www.youtube.com/watch?v=QapwrR9C3Z4,The impact of Covid-19 on media – rise of info...,"[is:youtube, is:webinar, disinformation, misin...",Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.youtube.com/watch?v=Q...,{'title': ['The impact of Covid-19 on media – ...,{'html': 'https://hypothes.is/a/oUZLeBo0Eeutht...,{'display_name': None},False,False
3,gHTeKBo0EeubG0_nm_NNpQ,2020-10-29T22:17:15.887213+00:00,2020-10-29T22:17:15.887213+00:00,acct:amyhcurtis@hypothes.is,https://www.youtube.com/watch?v=QapwrR9C3Z4,,[ann:summary],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.youtube.com/watch?v=Q...,{'title': ['The impact of Covid-19 on media – ...,{'html': 'https://hypothes.is/a/gHTeKBo0EeubG0...,{'display_name': None},False,False
4,fAZg8Bo0Eeu39uviYF17AQ,2020-10-29T22:17:08.501454+00:00,2020-10-29T22:17:08.501454+00:00,acct:amyhcurtis@hypothes.is,https://www.youtube.com/watch?v=QapwrR9C3Z4,,[ann:title],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.youtube.com/watch?v=Q...,{'title': ['The impact of Covid-19 on media – ...,{'html': 'https://hypothes.is/a/fAZg8Bo0Eeu39u...,{'display_name': None},False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44263,IqqGzNZdEeu7qW8TVqQlWA,2021-06-26T09:01:46.663954+00:00,2021-06-26T09:01:46.663954+00:00,acct:lucyparfitt16@hypothes.is,https://poseidon01.ssrn.com/delivery.php?ID=05...,2021-03-31,[has:date],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://poseidon01.ssrn.com/deliv...,{'title': ['Microsoft Word - Parry et al.docx']},{'html': 'https://hypothes.is/a/IqqGzNZdEeu7qW...,{'display_name': None},False,False
44264,GzcHitZdEeu3utsN1zIlXA,2021-06-26T09:01:34.146545+00:00,2021-06-26T09:01:34.146545+00:00,acct:lucyparfitt16@hypothes.is,https://poseidon01.ssrn.com/delivery.php?ID=05...,"Parry, H. M., Tut, G., Faustini, S., Stephens,...","[is:preprint, lang:en, COVID-19, vaccine, immu...",Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://poseidon01.ssrn.com/deliv...,{'title': ['Microsoft Word - Parry et al.docx']},{'html': 'https://hypothes.is/a/GzcHitZdEeu3ut...,{'display_name': None},False,False
44265,tkTclNZcEeu7dhcV5i6TGA,2021-06-26T08:58:44.804883+00:00,2021-06-26T08:58:44.804883+00:00,acct:lucyparfitt16@hypothes.is,https://poseidon01.ssrn.com/delivery.php?ID=05...,,[ann:summary],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://poseidon01.ssrn.com/deliv...,{'title': ['Microsoft Word - Parry et al.docx']},{'html': 'https://hypothes.is/a/tkTclNZcEeu7dh...,{'display_name': None},False,False
44266,p5oL9tZcEeuXK79Bg_WtoA,2021-06-26T08:58:20.188404+00:00,2021-06-26T08:58:20.188404+00:00,acct:lucyparfitt16@hypothes.is,https://poseidon01.ssrn.com/delivery.php?

In [16]:
# what is inside of our dataset?
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44268 entries, 0 to 44267
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           44268 non-null  object
 1   created      44268 non-null  object
 2   updated      44268 non-null  object
 3   user         44268 non-null  object
 4   uri          44268 non-null  object
 5   text         44268 non-null  object
 6   tags         44268 non-null  object
 7   group        44268 non-null  object
 8   permissions  44268 non-null  object
 9   target       44268 non-null  object
 10  document     44268 non-null  object
 11  links        44268 non-null  object
 12  user_info    44268 non-null  object
 13  flagged      44268 non-null  bool  
 14  hidden       44268 non-null  bool  
dtypes: bool(2), object(13)
memory usage: 4.5+ MB


### 2. Data exploration + curation

What are interesting insights to extract and what fields might need corrections?


In [17]:
# Get one sample, which is one annotation.
sample = df.sample(n=1)

In [18]:
sample.iloc[0].to_dict()

{'id': 'Vysrml8pEeyqqPcXjLc39A',
 'created': '2021-12-17T11:06:10.203673+00:00',
 'updated': '2021-12-17T11:06:10.203673+00:00',
 'user': 'acct:lucyparfitt16@hypothes.is',
 'uri': 'https://vancouversun.com/news/local-news/227-b-c-chiropractors-threaten-to-sue-regulatory-college-if-vaccines-ordered',
 'text': '',
 'tags': ['ann:summary'],
 'group': 'Jk8bYJdN',
 'permissions': {'read': ['group:__world__'],
  'admin': ['acct:lucyparfitt16@hypothes.is'],
  'update': ['acct:lucyparfitt16@hypothes.is'],
  'delete': ['acct:lucyparfitt16@hypothes.is']},
 'target': [{'source': 'https://vancouversun.com/news/local-news/227-b-c-chiropractors-threaten-to-sue-regulatory-college-if-vaccines-ordered',
   'selector': [{'type': 'RangeSelector',
     'endOffset': 177,
     'startOffset': 1,
     'endContainer': '/main[1]/article[1]/header[1]/p[1]',
     'startContainer': '/main[1]/article[1]/header[1]/p[1]'},
    {'end': 4504, 'type': 'TextPositionSelector', 'start': 4328},
    {'type': 'TextQuoteSelect

**Observation**: "text" seems to be missing in many cases, perhaps it is included in another column such as target

In [20]:
# Checking if the values of 'text' are reliable.
df['text'].value_counts()[:10]

              21698
2020-05         113
2020-07          75
2020-06          72
2020-04          66
2021-11-26       43
2020-04-30       35
2020-08          32
2020-05-06       30
2020-05-05       30
Name: text, dtype: int64

In [21]:
df.query('text != ""')[['id', 'text']]

,id,text
0,rnlcIho0EeuKgFfKPBbajQ,https://twitter.com/i/web/status/1306171100544...
1,qHdm3ho0EeuTJufnuQnIqQ,2020-09-16
2,oUZLeBo0EeuthtuX8fq6yQ,The impact of Covid-19 on media – rise of info...
5,uAs1rhozEeuVV7-ike6oNw,"Online Research Tools and Techniques. (2020, ..."
6,FaDHPhozEeuXjAsKbE0ErA,https://twitter.com/i/web/status/1305920211992...
...,...,...
44259,nXwluNZdEeu3vcv8gBb4GQ,2021-04-01
44260,lxgYJtZdEeuQ8F-gZ-Ae9w,Long Covid: Snapshot poll finds more than 1m p...
44263,IqqGzNZdEeu7qW8TVqQlWA,2021-03-31
44264,GzcHitZdEeu3utsN1zIlXA,"Parry, H. M., Tut, G., Faustini, S., Stephens,..."


**Check**: the total number of tags and the unique number of them.

In [22]:
all_tags = df.tags.sum()

In [23]:
len(all_tags)

145633

In [24]:
# Counting hashable objects
from collections import Counter

In [25]:
unique_tags = Counter(all_tags)

In [26]:
len(unique_tags) # Total number of unique tags existing in the dataset.

16474

In [27]:
unique_tags

Counter({'has:context': 5401,
         'has:date': 7688,
         'is:youtube': 111,
         'is:webinar': 23,
         'disinformation': 85,
         'misinformation': 708,
         'online': 110,
         'technology': 98,
         'reliability': 37,
         'infodemic': 44,
         'conspiracy': 47,
         'healthcare': 245,
         'public health': 595,
         'COVID-19': 7364,
         'media': 153,
         'lang:en': 8534,
         'ann:summary': 7083,
         'ann:title': 6841,
         'research': 824,
         'ethics': 46,
         'methodology': 17,
         'data': 693,
         'funding': 101,
         'application': 26,
         'is:interview': 2,
         'response': 318,
         'consequence': 42,
         'global': 167,
         'science': 460,
         'community': 140,
         'history': 21,
         'judgement': 16,
         'vaccine': 1950,
         'economy': 292,
         'lottery': 3,
         'webinar': 127,
         'poster': 1,
         'publish':

**Check**: create the column with only terms and the annotation fields - is, has, ann, lang. 

In [28]:
def create_tag_columns(row):
    row['is:'] = []
    row['has:'] = []
    row['ann:'] = []
    row['lang:'] = []
    row['terms_tags'] = []
    for tag in row['tags']:
        #tags = tags.replace('[','').replace(']','').split(',')
        #for tag in tags:
        if 'is:' in tag:
            row['is:'].append(tag.split(':')[1])
        elif 'has:' in tag:
            row['has:'].append(tag.split(':')[1])
        elif 'ann:' in tag:
            row['ann:'].append(tag.split(':')[1])
        elif 'lang:' in tag:
            row['lang:'].append(tag.split(':')[1])
        else:
            row['terms_tags'].append(tag)
    return row

def create_type_columns(row):
    for col in ['is:', 'has:', 'ann:']:
        for val in row[col]:
            row[f"{col.replace(':','')}_{val}"] = True
    return row

In [29]:
df = df.apply(lambda row: create_tag_columns(row), axis=1)

In [30]:
# Check if the 'source' field in 'target' column is equal to 'uri'
df['target__source'] = df['target'].apply(lambda target: target[0]['source'])
df['source_is_uri'] = df.apply(lambda row: row['uri'] == row['target__source'], axis=1)

In [31]:
df.query('source_is_uri == False') 

,id,created,updated,user,uri,text,tags,group,permissions,target,...,user_info,flagged,hidden,is:,has:,ann:,lang:,terms_tags,target__source,source_is_uri


**Check**: Confirmed that 'source' is equal to 'uri'.

In [32]:
# What is the difference between 'target', 'uri', 'links'? 
print("target 0: ", df['target'][0], "\n")
print("uri 0:", df['uri'][0], "\n")
print("links 0:", df['links'][0])

df['links'][0]

target 0:  [{'source': 'https://www.youtube.com/watch?v=QapwrR9C3Z4'}] 

uri 0: https://www.youtube.com/watch?v=QapwrR9C3Z4 

links 0: {'html': 'https://hypothes.is/a/rnlcIho0EeuKgFfKPBbajQ', 'incontext': 'https://hyp.is/rnlcIho0EeuKgFfKPBbajQ/www.youtube.com/watch?v=QapwrR9C3Z4', 'json': 'https://hypothes.is/api/annotations/rnlcIho0EeuKgFfKPBbajQ'}


{'html': 'https://hypothes.is/a/rnlcIho0EeuKgFfKPBbajQ',
 'incontext': 'https://hyp.is/rnlcIho0EeuKgFfKPBbajQ/www.youtube.com/watch?v=QapwrR9C3Z4',
 'json': 'https://hypothes.is/api/annotations/rnlcIho0EeuKgFfKPBbajQ'}

**Check** : the url in source field and "uri" are the same. "link" is the link to the annotation in the knowledge base.

## 3. Data preprocessing
To make the integrated data structure, data should be checked and cleaned.

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44268 entries, 0 to 44267
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              44268 non-null  object
 1   created         44268 non-null  object
 2   updated         44268 non-null  object
 3   user            44268 non-null  object
 4   uri             44268 non-null  object
 5   text            44268 non-null  object
 6   tags            44268 non-null  object
 7   group           44268 non-null  object
 8   permissions     44268 non-null  object
 9   target          44268 non-null  object
 10  document        44268 non-null  object
 11  links           44268 non-null  object
 12  user_info       44268 non-null  object
 13  flagged         44268 non-null  bool  
 14  hidden          44268 non-null  bool  
 15  is:             44268 non-null  object
 16  has:            44268 non-null  object
 17  ann:            44268 non-null  object
 18  lang: 

In [35]:
# A function to generate id from another column
def generate_id(string: str):
    return hashlib.sha1(string.encode("utf-8")).hexdigest()

In [36]:
# Check if the column 'id' is the row 'id', that we can use as a key for the annotation rows.
len(df) == len(df['id'].unique())

True

In [38]:
# What is the example of the 'target' column.
df['target'][0]

[{'source': 'https://www.youtube.com/watch?v=QapwrR9C3Z4'}]

In [39]:
# Check if all rows in 'target' column have 'source' field.
for i in df['target']: 
    if i[0]['source'] == None: 
        print(i)                 # There is no output, every row has source field. 

**Observations**: There is no null value in our dataset, but the empty value exists as '{}'

In [40]:
df['document']

0        {'title': ['The impact of Covid-19 on media – ...
1        {'title': ['The impact of Covid-19 on media – ...
2        {'title': ['The impact of Covid-19 on media – ...
3        {'title': ['The impact of Covid-19 on media – ...
4        {'title': ['The impact of Covid-19 on media – ...
                               ...                        
44263     {'title': ['Microsoft Word - Parry et al.docx']}
44264     {'title': ['Microsoft Word - Parry et al.docx']}
44265     {'title': ['Microsoft Word - Parry et al.docx']}
44266     {'title': ['Microsoft Word - Parry et al.docx']}
44267    {'title': ['f4d9b9_fddbfb2a0c05461cb4bdce2892f...
Name: document, Length: 44268, dtype: object

In [ ]:
df['document'].value_counts()

In [43]:
# Check if 'document' column has only one field 'title'. 
count = 1
b = 0
for i in df['document']:
    if i.get("title") == None: 
        b = b + 1
    if len(i) > 1: 
        count = count + 1
print(b, ": the number of the documents without title.")
print(count, ": the maximum number of field in document values.")

112 : the number of the documents without title.
1 : the maximum number of field in document values.


**Check**: 'document' has only one field called 'title'.

In [44]:
# A function to generate new column for non-null valued 'text_' column 

def extract_text_from_target(row):
    if row["text"] != "":
        row["text_"] = row["text"]
    elif "selector" in row["target"][0]:
        row["text_"] = [s["exact"] for s in row["target"][0]["selector"] if s["type"] == "TextQuoteSelector"][0]
    else:
        row["text_"] = ""
    return row

In [45]:
df = df.apply(lambda row: extract_text_from_target(row), axis=1)

In [46]:
df[["text", "text_"]]

,text,text_
0,https://twitter.com/i/web/status/1306171100544...,https://twitter.com/i/web/status/1306171100544...
1,2020-09-16,2020-09-16
2,The impact of Covid-19 on media – rise of info...,The impact of Covid-19 on media – rise of info...
3,,"In recent years, disinformation and misinforma..."
4,,The impact of Covid-19 on media – rise of info...
...,...,...
44263,2021-03-31,2021-03-31
44264,"Parry, H. M., Tut, G., Faustini, S., Stephens,...","Parry, H. M., Tut, G., Faustini, S., Stephens,..."
44265,,BackgroundAge is the major risk factor for mor...
44266,,1BNT162b2 vaccination in people over 80 years ...


For the better usage, process the values of 'document' column from pandas.Series to string.

In [48]:
# Fill in the 'document' column value from the 'target' column field 'exact', convert dict -> str
def extract_text_from_document(row):
    if row["document"].get("title"):
        row["document"] = row["document"]["title"][0]
    elif "selector" in row["target"][0]:
        row["document"] = [s["exact"] for s in row["target"][0]["selector"] if s["type"] == "TextQuoteSelector"][0]
    else:
        row["document"] = ""
    return row

In [49]:
df = df.apply(lambda row: extract_text_from_document(row), axis = 1)

In [51]:
# Check the result
df['document'] 

0        The impact of Covid-19 on media – rise of info...
1        The impact of Covid-19 on media – rise of info...
2        The impact of Covid-19 on media – rise of info...
3        The impact of Covid-19 on media – rise of info...
4        The impact of Covid-19 on media – rise of info...
                               ...                        
44263                    Microsoft Word - Parry et al.docx
44264                    Microsoft Word - Parry et al.docx
44265                    Microsoft Word - Parry et al.docx
44266                    Microsoft Word - Parry et al.docx
44267          f4d9b9_fddbfb2a0c05461cb4bdce2892f3cad0.pdf
Name: document, Length: 44268, dtype: object

**Question**: Which column to use for the key of this dataset? 'document' or 'uri'?

In [522]:
# unique value of 'document' column and 'uri' have different number.
len(df['document'].unique()) == len(df['uri'].unique())
print(len(df['document'].unique()), ": Nr. unique value for 'document'")
print(len(df['uri'].unique()), ": Nr. unique value for uri")

7596 : Nr. unique value for 'document'
9001 : Nr. unique value for uri


**Observations**: More unique values in uri than document, decide to go with 'document'.

cf. "Why multiple URLs redirect to the same websites?"

https://moz.com/community/q/topic/58696/i-have-multiple-urls-that-redirect-to-the-same-website-is-this-an-issue/3


In [52]:
# Generate document id from 'document' values.
df['doc_id'] = df['document'].apply(lambda doc: generate_id(doc))

In [53]:
df.head(5)

,id,created,updated,user,uri,text,tags,group,permissions,target,...,hidden,is:,has:,ann:,lang:,terms_tags,target__source,source_is_uri,text_,doc_id
0,rnlcIho0EeuKgFfKPBbajQ,2020-10-29T22:18:33.169969+00:00,2020-10-29T22:18:33.169969+00:00,acct:amyhcurtis@hypothes.is,https://www.youtube.com/watch?v=QapwrR9C3Z4,https://twitter.com/i/web/status/1306171100544...,[has:context],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.youtube.com/watch?v=Q...,...,False,[],[context],[],[],[],https://www.youtube.com/watch?v=QapwrR9C3Z4,True,https://twitter.com/i/web/status/1306171100544...,aa6c8c407d3e5b746cc4ea9889baf7a28eafb181
1,qHdm3ho0EeuTJufnuQnIqQ,2020-10-29T22:18:23.010549+00:00,2020-10-29T22:18:23.010549+00:00,acct:amyhcurtis@hypothes.is,https://www.youtube.com/watch?v=QapwrR9C3Z4,2020-09-16,[has:date],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.youtube.com/watch?v=Q...,...,False,[],[date],[],[],[],https://www.youtube.com/watch?v=QapwrR9C3Z4,True,2020-09-16,aa6c8c407d3e5b746cc4ea9889baf7a28eafb181
2,oUZLeBo0EeuthtuX8fq6yQ,2020-10-29T22:18:11.003006+00:00,2020-10-29T22:18:11.003006+00:00,acct:amyhcurtis@hypothes.is,https://www.youtube.com/watch?v=QapwrR9C3Z4,The impact of Covid-19 on media – rise of info...,"[is:youtube, is:webinar, disinformation, misin...",Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.youtube.com/watch?v=Q...,...,False,"[youtube, webinar]",[],[],[en],"[disinformation, misinformation, online, techn...",https://www.youtube.com/watch?v=QapwrR9C3Z4,True,The impact of Covid-19 on media – rise of info...,aa6c8c407d3e5b746cc4ea9889baf7a28eafb181
3,gHTeKBo0EeubG0_nm_NNpQ,2020-10-29T22:17:15.887213+00:00,2020-10-29T22:17:15.887213+00:00,acct:amyhcurtis@hypothes.is,https://www.youtube.com/watch?v=QapwrR9C3Z4,,[ann:summary],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.youtube.com/watch?v=Q...,...,False,[],[],[summary],[],[],https://www.youtube.com/watch?v=QapwrR9C3Z4,True,"In recent years, disinformation and misinforma...",aa6c8c407d3e5b746cc4ea9889baf7a28eafb181
4,fAZg8Bo0Eeu39uviYF17AQ,2020-10-29T22:17:08.501454+00:00,2020-10-29T22:17:08.501454+00:00,acct:amyhcurtis@hypothes.is,https://www.youtube.com/watch?v=QapwrR9C3Z4,,[ann:title],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.youtube.com/watch?v=Q...,...,False,[],[],[title],[],[],https://www.youtube.com/watch?v=QapwrR9C3Z4,True,The impact of Covid-19 on media – rise of info...,aa6c8c407d3e5b746cc4ea9889baf7a28eafb181


In [54]:
# Check if the generated id is unique values of document column
len(df['doc_id'].unique()) == len(df['document'].unique())

True

In [55]:
# Get the sample using doc_id
mysample = df.query('doc_id == "4debd0b8816961d9ee0fb81d9fd47884a3d23346"')
mysample

,id,created,updated,user,uri,text,tags,group,permissions,target,...,hidden,is:,has:,ann:,lang:,terms_tags,target__source,source_is_uri,text_,doc_id
10074,GKTDUOxqEeqdGgO3KOC83Q,2020-09-01T15:45:01.004518+00:00,2020-09-01T15:45:01.004518+00:00,acct:gailelhalaby@hypothes.is,https://www.imperial.ac.uk/stories/intersectin...,https://twitter.com/ImperialMed/status/1300375...,[has:context],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.imperial.ac.uk/storie...,...,False,[],[context],[],[],[],https://www.imperial.ac.uk/stories/intersectin...,True,https://twitter.com/ImperialMed/status/1300375...,4debd0b8816961d9ee0fb81d9fd47884a3d23346
10075,-vsbTOxpEeq_kMdMttrjaA,2020-09-01T15:44:11.268133+00:00,2020-09-01T15:44:11.268133+00:00,acct:gailelhalaby@hypothes.is,https://www.imperial.ac.uk/stories/intersectin...,2020\n,[has:date-approx],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.imperial.ac.uk/storie...,...,False,[],[date-approx],[],[],[],https://www.imperial.ac.uk/stories/intersectin...,True,2020\n,4debd0b8816961d9ee0fb81d9fd47884a3d23346
10076,4ydJtOxpEeqb8PfadgIuVw,2020-09-01T15:43:31.179769+00:00,2020-09-01T15:43:31.179769+00:00,acct:gailelhalaby@hypothes.is,https://www.imperial.ac.uk/stories/intersectin...,Two intersecting pandemics. (n.d.). Retrieved ...,"[is:webpage, lang:en, racism, COVID-19, corona...",Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.imperial.ac.uk/storie...,...,False,[webpage],[],[],[en],"[racism, COVID-19, coronavirus, pandemic, ethn...",https://www.imperial.ac.uk/stories/intersectin...,True,Two intersecting pandemics. (n.d.). Retrieved ...,4debd0b8816961d9ee0fb81d9fd47884a3d23346
10077,AOIWzuxpEeqMhLOdtJbP9A,2020-09-01T15:37:11.588640+00:00,2020-09-01T15:37:11.588640+00:00,acct:gailelhalaby@hypothes.is,https://www.imperial.ac.uk/stories/intersectin...,,[ann:summary],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.imperial.ac.uk/storie...,...,False,[],[],[summary],[],[],https://www.imperial.ac.uk/stories/intersectin...,True,How COVID-19 is laying bare structural racism ...,4debd0b8816961d9ee0fb81d9fd47884a3d23346
10078,_ESrkOxoEeqZZ8M7psIcqA,2020-09-01T15:37:03.663208+00:00,2020-09-01T15:37:03.663208+00:00,acct:gailelhalaby@hypothes.is,https://www.imperial.ac.uk/stories/intersectin...,,[ann:title],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.imperial.ac.uk/storie...,...,False,[],[],[title],[],[],https://www.imperial.ac.uk/stories/intersectin...,True,Two intersecting pandemics,4debd0b8816961d9ee0fb81d9fd47884a3d23346


## 4. Data structure

There are three data structures to create from this pipeline. 

1. data from hypothesis as DataFrame -> each row is an annotation. To get a document, we need to groupby doc_id.
2. data from hypothesis transformed into "documents": a list of documents, where one document has a subfield "annotations", where annotations are. -> OUR REFERENCE
3. objects to ingest in Elastic Search: one document => one dict => one annotation -> we want to create these objects out of (2) 

**"Steps" to get the document defined above (after cleaning steps)**
1. Generate doc_id
2. Group dataset by doc_id
3. The top level of the document object comes from properties across rows (groupby) meaning that all the rows in that group have these same properties.
4. The object annotations contains all the info coming from individual rows within the group (1 row = 1 dict in the list)

### Generate document (2)

In [424]:
'''
2) OUR REFERENCE document format.

documents = [
    {
        "_id": "id",  # 
        "document_uri": "uri", #from df
        "document ": "document", # from df,
        "tags": [], # groupby uri -> concatenated lists of tags 
        "annotations": [ # groupby uri -> each row is dict for the columns "text", "tags", "target", "links"
            {
                "ann_id": "id from the row"
                "tags": []
                "target": [],
                "text": str,
                "links": dict
            }, {

            } 
        ]
    }
]
'''

'\n2) purpose data format\n\ndocuments = [\n    {\n        "_id": "id",  # \n        "document_uri": "uri", #from df\n        "document ": "document", # from df,\n        "tags": [], # groupby uri -> concatenated lists of tags \n        "annotations": [ # groupby uri -> each row is dict for the columns "text", "tags", "target", "links"\n            {\n                "ann_id": "id from the row"\n                "tags": []\n                "target": [],\n                "text": str,\n                "links": dict\n            }, {\n\n            } \n        ]\n    }\n]\n'

1. Original dataframe.

In [56]:
# Give the id column the name ann_id.
df = df.rename(columns={'id': 'ann_id'})

In [57]:
df.columns

Index(['ann_id', 'created', 'updated', 'user', 'uri', 'text', 'tags', 'group',
       'permissions', 'target', 'document', 'links', 'user_info', 'flagged',
       'hidden', 'is:', 'has:', 'ann:', 'lang:', 'terms_tags',
       'target__source', 'source_is_uri', 'text_', 'doc_id'],
      dtype='object')

In [58]:
# Delete the columns that were added during data exploration, and keep the original columns
# Generate multi-index dataframe, "df1" 
df = df.drop(columns = ["is:", "has:",  "ann:", "lang:", "source_is_uri", "target__source", "terms_tags"])
df1 = df.set_index(['doc_id', 'ann_id'])


In [59]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 44268 entries, ('aa6c8c407d3e5b746cc4ea9889baf7a28eafb181', 'rnlcIho0EeuKgFfKPBbajQ') to ('551363c260f71f9d57f2d08eb66a8e21a9aa2802', '-eARsNZZEeu-amPoGgaXSg')
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   created      44268 non-null  object
 1   updated      44268 non-null  object
 2   user         44268 non-null  object
 3   uri          44268 non-null  object
 4   text         44268 non-null  object
 5   tags         44268 non-null  object
 6   group        44268 non-null  object
 7   permissions  44268 non-null  object
 8   target       44268 non-null  object
 9   document     44268 non-null  object
 10  links        44268 non-null  object
 11  user_info    44268 non-null  object
 12  flagged      44268 non-null  bool  
 13  hidden       44268 non-null  bool  
 14  text_        44268 non-null  object
dtypes: bool(2), object(13)
memory usage: 6.4+ MB


In [60]:
df1.head(10)

created  \
doc_id                                   ann_id                                                     
aa6c8c407d3e5b746cc4ea9889baf7a28eafb181 rnlcIho0EeuKgFfKPBbajQ  2020-10-29T22:18:33.169969+00:00   
                                         qHdm3ho0EeuTJufnuQnIqQ  2020-10-29T22:18:23.010549+00:00   
                                         oUZLeBo0EeuthtuX8fq6yQ  2020-10-29T22:18:11.003006+00:00   
                                         gHTeKBo0EeubG0_nm_NNpQ  2020-10-29T22:17:15.887213+00:00   
                                         fAZg8Bo0Eeu39uviYF17AQ  2020-10-29T22:17:08.501454+00:00   
b0204dd848c1f25b0506d61889bff67127024f7f uAs1rhozEeuVV7-ike6oNw  2020-10-29T22:11:39.622491+00:00   
                                         FaDHPhozEeuXjAsKbE0ErA  2020-10-29T22:07:07.184512+00:00   
                                         EFakYBozEeuw3TO-taovEg  2020-10-29T22:06:58.254626+00:00   
                                         ChBWPBozEeuqI0dyN6mJpw  2020-10-29T22:06:47.797225+00:00   
                                         BQ6GDhozEeudISeH9k9VOA  2020-10-29T22:06:39.294849+00:00   

                                                                                          updated  \
doc_id                                   ann_id                                                     
aa6c8c407d3e5b746cc4ea9889baf7a28eafb181 rnlcIho0EeuKgFfKPBbajQ  2020-10-29T22:18:33.169969+00:00   
                                         qHdm3ho0EeuTJufnuQnIqQ  2020-10-29T22:18:23.010549+00:00   
                                         oUZLeBo0EeuthtuX8fq6yQ  2020-10-29T22:18:11.003006+00:00   
                                         gHTeKBo0EeubG0_nm_NNpQ  2020-10-29T22:17:15.887213+00:00   
                                         fAZg8Bo0Eeu39uviYF17AQ  2020-10-29T22:17:08.501454+00:00   
b0204dd848c1f25b0506d61889bff67127024f7f uAs1rhozEeuVV7-ike6oNw  2020-10-29T22:11:39.622491+00:00   
                                         FaDHPhozEeuXjAsKbE0ErA  2020-10-29T22:07:07.184512+00:00   
                                         EFakYBozEeuw3TO-taovEg  2020-10-29T22:06:58.254626+00:00   
                                         ChBWPBozEeuqI0dyN6mJpw  2020-10-29T22:06:47.797225+00:00   
                                         BQ6GDhozEeudISeH9k9VOA  2020-10-29T22:06:39.294849+00:00   

                                                                                        user  \
doc_id                                   ann_id                                                
aa6c8c407d3e5b746cc4ea9889baf7a28eafb181 rnlcIho0EeuKgFfKPBbajQ  acct:amyhcurtis@hypothes.is   
                                         qHdm3ho0EeuTJufnuQnIqQ  acct:amyhcurtis@hypothes.is   
                                         oUZLeBo0EeuthtuX8fq6yQ  acct:amyhcurtis@hypothes.is   
                                         gHTeKBo0EeubG0_nm_NNpQ  acct:amyhcurtis@hypothes.is   
                                         fAZg8Bo0Eeu39uviYF17AQ  acct:amyhcurtis@hypothes.is   
b0204dd848c1f25b0506d61889bff67127024f7f uAs1rhozEeuVV7-ike6oNw  acct:amyhcurtis@hypothes.is   
                                         FaDHPhozEeuXjAsKbE0ErA  acct:amyhcurtis@hypothes.is   
                                         EFakYBozEeuw3TO-taovEg  acct:amyhcurtis@hypothes.is   
                                         ChBWPBozEeuqI0dyN6mJpw  acct:amyhcurtis@hypothes.is   
                                         BQ6GDhozEeudISeH9k9VOA  acct:amyhcurtis@hypothes.is   

                                                                                                         uri  \
doc_id                                   ann_id                                                                
aa6c8c407d3e5b746cc4ea9889baf7a28eafb181 rnlcIho0EeuKgFfKPBbajQ  https://www.youtube.com/watch?v=QapwrR9C3Z4   
                                         qHdm3ho0EeuTJufnuQnIqQ  https://www.youtube.com/watch?v=QapwrR9C3Z4   
                                         oUZLeBo0EeuthtuX8fq6yQ

2. Define a function to generate our reference document format.

In [61]:
def generate_document(df: pd.DataFrame) -> dict:
    
    """
    :param df: This dataframe is actually a "grouped-by" dataframe, meaning that doc_id, uri, document and tags are the same for all rows
    :return:
    """
    
    assert len(df['doc_id'].unique()) == 1
    assert len(df['document'].unique()) == 1

    document = {
        "_id": df['doc_id'].iloc[0],
        "document_uri": df["uri"].iloc[0],
        "document": df['document'].iloc[0],
        "tags": [],
        "annotations": [], 
        "created": df['created'].iloc[0], 
        "updated": df['updated'].iloc[0], 
        "user": df['user'].iloc[0], 
        "group": df['group'].iloc[0], 
        "permissions": df['permissions'].iloc[0], 
        "user_info": df['user_info'].iloc[0], 
        "flagged": df['flagged'].iloc[0], 
        "hidden": df['hidden'].iloc[0]
        
    }

    for i, row in df.iterrows():
        annotation = {}
        # this is at level of document (first level)
        document["tags"] += row["tags"]

        # This is at level of annotation (second level)
        annotation["text"] =  row["text_"]
        annotation["tags"] = row["tags"]
        annotation["ann_id"] = row["ann_id"]
        annotation["target"] = row["target"]
        annotation["links"] = row["links"]
        document["annotations"].append(annotation)

    return document


In [62]:
df.columns

Index(['ann_id', 'created', 'updated', 'user', 'uri', 'text', 'tags', 'group',
       'permissions', 'target', 'document', 'links', 'user_info', 'flagged',
       'hidden', 'text_', 'doc_id'],
      dtype='object')

In [63]:
# Generate documents format from dataframe. 
documents = []

for doc_id in df['doc_id'].unique():
    sub_df = df.query("doc_id == @doc_id")
    documents.append(generate_document(sub_df))


In [64]:
# How many document exist in the list documents?
len(documents)

7596

In [66]:
df.head(5)

,ann_id,created,updated,user,uri,text,tags,group,permissions,target,document,links,user_info,flagged,hidden,text_,doc_id
0,rnlcIho0EeuKgFfKPBbajQ,2020-10-29T22:18:33.169969+00:00,2020-10-29T22:18:33.169969+00:00,acct:amyhcurtis@hypothes.is,https://www.youtube.com/watch?v=QapwrR9C3Z4,https://twitter.com/i/web/status/1306171100544...,[has:context],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.youtube.com/watch?v=Q...,The impact of Covid-19 on media – rise of info...,{'html': 'https://hypothes.is/a/rnlcIho0EeuKgF...,{'display_name': None},False,False,https://twitter.com/i/web/status/1306171100544...,aa6c8c407d3e5b746cc4ea9889baf7a28eafb181
1,qHdm3ho0EeuTJufnuQnIqQ,2020-10-29T22:18:23.010549+00:00,2020-10-29T22:18:23.010549+00:00,acct:amyhcurtis@hypothes.is,https://www.youtube.com/watch?v=QapwrR9C3Z4,2020-09-16,[has:date],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.youtube.com/watch?v=Q...,The impact of Covid-19 on media – rise of info...,{'html': 'https://hypothes.is/a/qHdm3ho0EeuTJu...,{'display_name': None},False,False,2020-09-16,aa6c8c407d3e5b746cc4ea9889baf7a28eafb181
2,oUZLeBo0EeuthtuX8fq6yQ,2020-10-29T22:18:11.003006+00:00,2020-10-29T22:18:11.003006+00:00,acct:amyhcurtis@hypothes.is,https://www.youtube.com/watch?v=QapwrR9C3Z4,The impact of Covid-19 on media – rise of info...,"[is:youtube, is:webinar, disinformation, misin...",Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.youtube.com/watch?v=Q...,The impact of Covid-19 on media – rise of info...,{'html': 'https://hypothes.is/a/oUZLeBo0Eeutht...,{'display_name': None},False,False,The impact of Covid-19 on media – rise of info...,aa6c8c407d3e5b746cc4ea9889baf7a28eafb181
3,gHTeKBo0EeubG0_nm_NNpQ,2020-10-29T22:17:15.887213+00:00,2020-10-29T22:17:15.887213+00:00,acct:amyhcurtis@hypothes.is,https://www.youtube.com/watch?v=QapwrR9C3Z4,,[ann:summary],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.youtube.com/watch?v=Q...,The impact of Covid-19 on media – rise of info...,{'html': 'https://hypothes.is/a/gHTeKBo0EeubG0...,{'display_name': None},False,False,"In recent years, disinformation and misinforma...",aa6c8c407d3e5b746cc4ea9889baf7a28eafb181
4,fAZg8Bo0Eeu39uviYF17AQ,2020-10-29T22:17:08.501454+00:00,2020-10-29T22:17:08.501454+00:00,acct:amyhcurtis@hypothes.is,https://www.youtube.com/watch?v=QapwrR9C3Z4,,[ann:title],Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.youtube.com/watch?v=Q...,The impact of Covid-19 on media – rise of info...,{'html': 'https://hypothes.is/a/fAZg8Bo0Eeu39u...,{'display_name': None},False,False,The impact of Covid-19 on media – rise of info...,aa6c8c407d3e5b746cc4ea9889baf7a28eafb181


3. Create json file out of the documents 

In [68]:
import json
with open('hypothesis_documents_v1.jsonl', 'w') as f:
    for doc in documents:
        try:
            f.write(json.dumps(doc, ensure_ascii=False, default = str) + '\n')
        except:
            print(doc)
            raise Exception

### Generate document (3) to ingest into ElasticSearch


document_es = {

    "_id": doc_id + "_" + ann_id, (composition of original doc_id and ann_id)
    "parent_doc_id": doc_id, (the id of the document to which this annotation belongs)
    "document_uri": as they are in the original document
    "document": as they are in the original document
    "ann_id": annotations["ann_id"],
    "tags": annotations["tags"], (not tags from the first level, for that specific annotation)
    rest of the fields of that annotation as they are
    ...

}




In [69]:
df.columns

Index(['ann_id', 'created', 'updated', 'user', 'uri', 'text', 'tags', 'group',
       'permissions', 'target', 'document', 'links', 'user_info', 'flagged',
       'hidden', 'text_', 'doc_id'],
      dtype='object')

In [70]:
# function to generate the 3rd data format.
def generate_document3(df: pd.DataFrame) -> dict:
    """
    
    :param df: This dataframe is actually a "grouped-by" dataframe, meaning that doc_id, uri, document and tags are the same for all rows
    :return:
    
    """
    assert len(df['doc_id'].unique()) == 1
    assert len(df['document'].unique()) == 1

    document = {
        "ann_id": df['doc_id'].iloc[0] + "_" + df['ann_id'].iloc[0],
        "parent_doc_id": df['doc_id'].iloc[0],
        "document_uri": df["uri"].iloc[0],
        "document": df['document'].iloc[0],
        "tags": df['tags'].iloc[0], 
        "created": df['created'].iloc[0], 
        "updated": df['updated'].iloc[0], 
        "user": df["user"].iloc[0],
        "text": df["text"].iloc[0],
        "group": df["group"].iloc[0], 
        "permissions": df["permissions"].iloc[0], 
        "target": df["target"].iloc[0], 
        "links": df["links"].iloc[0], 
        "user_info": df["user_info"].iloc[0], 
        "flagged": df["flagged"].iloc[0], 
        "hidden": df["hidden"].iloc[0], 
        
    }

    
    return document


In [71]:
documents_es = []

for doc_id in df['doc_id'].unique():
    sub_df = df.query("doc_id == @doc_id")
    documents_es.append(generate_document3(sub_df))


In [72]:
documents_es[:5]

[{'ann_id': 'aa6c8c407d3e5b746cc4ea9889baf7a28eafb181_rnlcIho0EeuKgFfKPBbajQ',
  'parent_doc_id': 'aa6c8c407d3e5b746cc4ea9889baf7a28eafb181',
  'document_uri': 'https://www.youtube.com/watch?v=QapwrR9C3Z4',
  'document': 'The impact of Covid-19 on media – rise of infodemics?',
  'tags': ['has:context'],
  'created': '2020-10-29T22:18:33.169969+00:00',
  'updated': '2020-10-29T22:18:33.169969+00:00',
  'user': 'acct:amyhcurtis@hypothes.is',
  'text': 'https://twitter.com/i/web/status/1306171100544602112',
  'group': 'Jk8bYJdN',
  'permissions': {'read': ['group:__world__'],
   'admin': ['acct:amyhcurtis@hypothes.is'],
   'update': ['acct:amyhcurtis@hypothes.is'],
   'delete': ['acct:amyhcurtis@hypothes.is']},
  'target': [{'source': 'https://www.youtube.com/watch?v=QapwrR9C3Z4'}],
  'links': {'html': 'https://hypothes.is/a/rnlcIho0EeuKgFfKPBbajQ',
   'incontext': 'https://hyp.is/rnlcIho0EeuKgFfKPBbajQ/www.youtube.com/watch?v=QapwrR9C3Z4',
   'json': 'https://hypothes.is/api/annotations/

In [73]:
len(documents_es)

7596

In [74]:
# Create json file out of the document
import json
with open('hypothesis_documents_v2.jsonl', 'w') as f:
    for doc in documents_es:
        try:
            f.write(json.dumps(doc, ensure_ascii=False, default = str) + '\n')
        except:
            print(doc)
            raise Exception

## 5. Search by Elasticsearch with our Dataset

In [ ]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
es.info().body

### Object to ingest into Elastic Search

In [668]:
df3 = (
    pd.read_json('hypothesis_documents_v2.jsonl', orient='records', lines=True)
    .dropna()
    .sample(5000, random_state=42)
    .reset_index()
)

In [647]:
df3

,index,ann_id,parent_doc_id,document_uri,document,tags,created,updated,user,text,group,permissions,target,links,user_info,flagged,hidden
0,1261,1d6c04a561b7308e70998b00d262798d515a52f7_Y2qaa...,1d6c04a561b7308e70998b00d262798d515a52f7,https://www.youtube.com/watch?v=0pQeCANMzss,FEPS COVID Response Webinar: EU Spending that ...,[has:context],2020-07-13T10:48:29.501731+00:00,2020-07-13T10:48:29.501731+00:00,acct:edampf@hypothes.is,https://twitter.com/i/web/status/1253304970683...,Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.youtube.com/watch?v=0...,{'html': 'https://hypothes.is/a/Y2qaaMT2EeqMN3...,{'display_name': None},False,False
1,7417,43eb9ac5ef74a34a37fdcae5f2ee9de85e7e7ae5_zkfBn...,43eb9ac5ef74a34a37fdcae5f2ee9de85e7e7ae5,https://miami.cbslocal.com/2021/08/09/ron-desa...,Gov. Ron DeSantis’ Office: State Education Boa...,[has:context],2021-08-12T13:11:28.970286+00:00,2021-08-12T13:11:28.970286+00:00,acct:lucyparfitt16@hypothes.is,https://twitter.com/CBSMiami/status/1424825696...,Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://miami.cbslocal.com/2021/0...,{'html': 'https://hypothes.is/a/zkfBnvtuEeu8rn...,{'display_name': None},False,False
2,6073,a3be7436686d3bb6fd1f1e9ab1dc036e9fd6aa43_KL5N5...,a3be7436686d3bb6fd1f1e9ab1dc036e9fd6aa43,https://www.al.com/news/2021/07/im-sorry-but-i...,"‘I’m sorry, but it’s too late’: Alabama doctor...","[is:webpage, lang:en, COVID-19, vaccine, vacci...",2021-08-04T15:46:39.989240+00:00,2021-08-04T15:46:39.989240+00:00,acct:zoe_ikeotuonye@hypothes.is,"‘I’m sorry, but it’s too late’: Alabama doctor...",Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.al.com/news/2021/07/i...,{'html': 'https://hypothes.is/a/KL5N5PU7EeuZZq...,{'display_name': None},False,False
3,3047,6f9f99c24561252cf219ae7288d5d9c5ddc1e8f9_qxvVF...,6f9f99c24561252cf219ae7288d5d9c5ddc1e8f9,https://twitter.com/JedKolko/status/1301865629...,Jed Kolko on Twitter,[has:date],2020-09-07T13:25:59.429127+00:00,2020-09-07T13:25:59.429127+00:00,acct:ErikStuchly@hypothes.is,2020-09-04,Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://twitter.com/JedKolko/stat...,{'html': 'https://hypothes.is/a/qxvVFPENEeqZyk...,{'display_name': None},False,False
4,586,44f042f37f0123316b93da33c07cd6fad350ac85_fDFir...,44f042f37f0123316b93da33c07cd6fad350ac85,https://www.reddit.com/r/BehSciMeta/comments/g...,r/BehSciMeta - Comment by u/UHahn on ”What is ...,"[is:reddit, lang:en, impact, retraction, scien...",2020-06-17T15:37:43.186311+00:00,2020-06-17T15:37:43.186311+00:00,acct:Marlene_Wulf@hypothes.is,r/BehSciMeta—Comment by u/UHahn on ”What is th...,Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.reddit.com/r/BehSciMe...,{'html': 'https://hypothes.is/a/fDFirLCwEeqBRw...,{'display_name': None},False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,3724,9b2d22989c399aa4d6d27b2edb91434e3a8562a1_lFm2i...,9b2d22989c399aa4d6d27b2edb91434e3a8562a1,https://www.nature.com/articles/d41586-021-001...,How epidemiology has shaped the COVID pandemic,"[lang:en, epidemiology, COVID-19, pandemic, re...",2021-01-28T14:17:40.188166+00:00,2021-01-28T14:17:40.188166+00:00,acct:david_gru@hypothes.is,(2021). How epidemiology has shaped the COVID ...,Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.nature.com/articles/d...,{'html': 'https://hypothes.is/a/lFm2imFzEeuzCC...,{'display_name': None},False,False
4996,4187,99096118e61066d9764f05ae6a2cde3880fbd247_FnMQt...,99096118e61066d9764f05ae6a2cde3880fbd247,https://today.law.harvard.edu/waiving-covid-va...,Waiving COVID vaccine patent rights? It's comp...,[has:date],2021-05-11T20:40:14.057647+00:00,2021-05-11T20:40:14.057647+00:00,acct:sophia.sterckx@hypothes.is,04-05-21,Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://today.law.harvard.edu/wai...,{'ht

In [648]:
df3.columns

Index(['index', 'ann_id', 'parent_doc_id', 'document_uri', 'document', 'tags',
       'created', 'updated', 'user', 'text', 'group', 'permissions', 'target',
       'links', 'user_info', 'flagged', 'hidden'],
      dtype='object')

In [649]:
df3 = df3.drop(['index'], axis = 1)

In [650]:
df3.head(5)

,ann_id,parent_doc_id,document_uri,document,tags,created,updated,user,text,group,permissions,target,links,user_info,flagged,hidden
0,1d6c04a561b7308e70998b00d262798d515a52f7_Y2qaa...,1d6c04a561b7308e70998b00d262798d515a52f7,https://www.youtube.com/watch?v=0pQeCANMzss,FEPS COVID Response Webinar: EU Spending that ...,[has:context],2020-07-13T10:48:29.501731+00:00,2020-07-13T10:48:29.501731+00:00,acct:edampf@hypothes.is,https://twitter.com/i/web/status/1253304970683...,Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.youtube.com/watch?v=0...,{'html': 'https://hypothes.is/a/Y2qaaMT2EeqMN3...,{'display_name': None},False,False
1,43eb9ac5ef74a34a37fdcae5f2ee9de85e7e7ae5_zkfBn...,43eb9ac5ef74a34a37fdcae5f2ee9de85e7e7ae5,https://miami.cbslocal.com/2021/08/09/ron-desa...,Gov. Ron DeSantis’ Office: State Education Boa...,[has:context],2021-08-12T13:11:28.970286+00:00,2021-08-12T13:11:28.970286+00:00,acct:lucyparfitt16@hypothes.is,https://twitter.com/CBSMiami/status/1424825696...,Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://miami.cbslocal.com/2021/0...,{'html': 'https://hypothes.is/a/zkfBnvtuEeu8rn...,{'display_name': None},False,False
2,a3be7436686d3bb6fd1f1e9ab1dc036e9fd6aa43_KL5N5...,a3be7436686d3bb6fd1f1e9ab1dc036e9fd6aa43,https://www.al.com/news/2021/07/im-sorry-but-i...,"‘I’m sorry, but it’s too late’: Alabama doctor...","[is:webpage, lang:en, COVID-19, vaccine, vacci...",2021-08-04T15:46:39.989240+00:00,2021-08-04T15:46:39.989240+00:00,acct:zoe_ikeotuonye@hypothes.is,"‘I’m sorry, but it’s too late’: Alabama doctor...",Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.al.com/news/2021/07/i...,{'html': 'https://hypothes.is/a/KL5N5PU7EeuZZq...,{'display_name': None},False,False
3,6f9f99c24561252cf219ae7288d5d9c5ddc1e8f9_qxvVF...,6f9f99c24561252cf219ae7288d5d9c5ddc1e8f9,https://twitter.com/JedKolko/status/1301865629...,Jed Kolko on Twitter,[has:date],2020-09-07T13:25:59.429127+00:00,2020-09-07T13:25:59.429127+00:00,acct:ErikStuchly@hypothes.is,2020-09-04,Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://twitter.com/JedKolko/stat...,{'html': 'https://hypothes.is/a/qxvVFPENEeqZyk...,{'display_name': None},False,False
4,44f042f37f0123316b93da33c07cd6fad350ac85_fDFir...,44f042f37f0123316b93da33c07cd6fad350ac85,https://www.reddit.com/r/BehSciMeta/comments/g...,r/BehSciMeta - Comment by u/UHahn on ”What is ...,"[is:reddit, lang:en, impact, retraction, scien...",2020-06-17T15:37:43.186311+00:00,2020-06-17T15:37:43.186311+00:00,acct:Marlene_Wulf@hypothes.is,r/BehSciMeta—Comment by u/UHahn on ”What is th...,Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://www.reddit.com/r/BehSciMe...,{'html': 'https://hypothes.is/a/fDFirLCwEeqBRw...,{'display_name': None},False,False


### 1. Creating index to use in Elastic search

In [681]:
# if running this code for the first time, untoggle the last line, so that you can "create" index.

mappings = {
    
    "properties": {
        
        "ann_id" : {"type": "text", "analyzer": "standard"}, 
        "parent_doc_id" : {"type": "text", "analyzer": "standard"}, 
        "document_uri" : {"type": "text", "analyzer": "standard"}, 
        "document" : {"type": "text", "analyzer": "standard"}, 
        "tags": {"type": "keyword"},
        "created" : {"type": "date"}, 
        "updated": {"type": "date"}, 
        "user": {"type": "text", "analyzer": "standard"}, 
        "text" : {"type": "text", "analyzer": "standard"}, 
        "group": {"type": "text", "analyzer": "standard"}, 
        "permissions": {"type": "nested"}, 
        "target": {"type": "nested"}, 
        "links": {"type": "nested"}, 
        "user_info": {"type": "object"}, 
        "flagged": {"type": "text", "analyzer": "standard"}, 
        "hidden": {"type": "text", "analyzer": "standard"}
        
    
    }
}

# es.indices.create(index= "hypothesis_v1", mappings = mappings)

In [670]:
df3["user_info"].iloc[90]

{'display_name': None}

### Assigning field data types

1. text: general fields belong here, and analyzer parameter also should be specified e.g.standard, english, french..
2. keyword:  used for structured content such as IDs, email addresses, hostnames, status codes, zip codes, or tags.
3. date: logged data column belongs here.
4. nested: it is an array of objects fields, where object is a json data. Json objects store data hierarchically. Those columns with inner json objects belong here.
5. OTHER data type has not been used, however, it needs to be discussed later.

### 2. Add data to the index created above

In [671]:
from elasticsearch.helpers import bulk

bulk_data = []
for i,row in df3.iterrows():
    bulk_data.append(
        {
            "_index": "hypothesis_v1",
            "_id": row['ann_id'],
            "_source": {
                "parent_doc_id": row["parent_doc_id"],
                "document_uri": row["document_uri"],
                "document": row["document"], 
                "tags": row["tags"], 
                "created": row["created"], 
                "updated": row["updated"], 
                "user": row["user"], 
                "text": row["text"], 
                "group": row["group"], 
                "permissions": row["permissions"], 
                "target": row["target"], 
                "links": row["links"], 
                "user_info": row["user_info"], 
                "flagged": row["flagged"], 
                "hidden": row["hidden"]               
                
            }
        }
    )
bulk(es, bulk_data)

(5000, [])

In [672]:
es.indices.refresh(index="hypothesis_v1")
es.cat.count(index="hypothesis_v1", format="json")

ListApiResponse([{'epoch': '1680528881', 'timestamp': '13:34:41', 'count': '5000'}])

In [673]:
sample = df3.sample(n=1)

### 3. Search the data with Elastic Search

1. Search annotation with the ann_id given

In [674]:
resp = es.search(
    index="hypothesis_v1",
    body={
        "query": {
            "bool": {
                "must": {
                    "match_phrase": {
                        "_id": "4011af8ea429e3c113c7328a721f6a2af2fd188f_L5lt6s5MEeqm_pesYHJVVQ",
                    }
                },
                },
        },            
    }
)
resp

/var/folders/j0/9xtsbl9502ng_j6cfbbp6fq00000gn/T/ipykernel_92391/630335297.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  resp = es.search(


ObjectApiResponse({'took': 5, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'hypothesis_v1', '_id': '4011af8ea429e3c113c7328a721f6a2af2fd188f_L5lt6s5MEeqm_pesYHJVVQ', '_score': 1.0, '_source': {'parent_doc_id': '4011af8ea429e3c113c7328a721f6a2af2fd188f', 'document_uri': 'https://osf.io/preprints/socarxiv/b5zae/', 'document': 'COVID-19_Insights from Innovation Economists', 'tags': ['has:date'], 'created': '2020-07-25T07:55:19.812600+00:00', 'updated': '2020-07-25T07:55:19.812600+00:00', 'user': 'acct:ErikStuchly@hypothes.is', 'text': '2020-04-14', 'group': 'Jk8bYJdN', 'permissions': {'read': ['group:__world__'], 'admin': ['acct:ErikStuchly@hypothes.is'], 'update': ['acct:ErikStuchly@hypothes.is'], 'delete': ['acct:ErikStuchly@hypothes.is']}, 'target': [{'source': 'https://osf.io/preprints/socarxiv/b5zae/'}], 'links': {'html': 'https://hypothes.is/a/L5lt6s5ME

In [675]:
sample

,index,ann_id,parent_doc_id,document_uri,document,tags,created,updated,user,text,group,permissions,target,links,user_info,flagged,hidden
3386,2584,922ebc8a3532ef32200d2e8b53c0772f401eaf9f_JEU9V...,922ebc8a3532ef32200d2e8b53c0772f401eaf9f,https://arxiv.org/abs/2008.02413,Impact of COVID-19 on Public Transit Accessibi...,"[is:preprint, lang:en, COVID-19, accessibility...",2020-08-10T14:24:03.576045+00:00,2021-03-19T12:17:54.281335+00:00,acct:SIYANYE@hypothes.is,"Wilbur, M., Ayman, A., Ouyang, A., Poon, V., K...",Jk8bYJdN,"{'read': ['group:__world__'], 'admin': ['acct:...",[{'source': 'https://arxiv.org/abs/2008.02413'}],{'html': 'https://hypothes.is/a/JEU9VNsVEeqKJk...,{'display_name': 'JessieYe'},False,False


2. Search annotation with the given condition.

In [678]:
# Search user, document value for the document title 'Two instersecting pandemics'
res = es.search(index="hypothesis_v1", body={"query": {"match": {"user": "acct:SIYANYE@hypothes.is"}}})
for doc in res['hits']['hits']:
    print("%s) %s" % (doc['_source']['user'], doc['_source']['document']))

acct:SIYANYE@hypothes.is) What makes anti-vaccine websites persuasive? A content analysis of techniques used by anti-vaccine websites to engender anti-vaccine sentiment
acct:SIYANYE@hypothes.is) Coronavirus vaccine safe in early trial, hydroxychloroquine may increase death risk
acct:SIYANYE@hypothes.is) The Age-Old Struggle against the Antivaccinationists | NEJM
acct:SIYANYE@hypothes.is) Immunization in the United States: Recommendations, Barriers, and Measures to Improve Compliance: Part 1: Childhood Vaccinations
acct:SIYANYE@hypothes.is) An increase in people dying at home suggests coronavirus deaths in Houston may be higher than reported
acct:SIYANYE@hypothes.is) When should British schools reopen? Here's what the science tells us | Devi Sridhar and Ines Hassan
acct:SIYANYE@hypothes.is) Benjy Renton on Twitter
acct:SIYANYE@hypothes.is) Parents' and informal caregivers' views and experiences of communication about routine childhood vaccination: a synthesis of qualitative evidence
acc

/var/folders/j0/9xtsbl9502ng_j6cfbbp6fq00000gn/T/ipykernel_92391/3877528631.py:2: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index="hypothesis_v1", body={"query": {"match": {"user": "acct:SIYANYE@hypothes.is"}}})


### 4. Update, delete data and delete indices in ElasticSearch

In progress.

In [680]:
# Delete data from the search engine.
es.delete(index="hypothesis_v1", id="4011af8ea429e3c113c7328a721f6a2af2fd188f_L5lt6s5MEeqm_pesYHJVVQ")

ObjectApiResponse({'_index': 'hypothesis_v1', '_id': '4011af8ea429e3c113c7328a721f6a2af2fd188f_L5lt6s5MEeqm_pesYHJVVQ', '_version': 2, 'result': 'deleted', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 5000, '_primary_term': 1})

In [679]:
# Update data from the search engine
# es is equal to Elasticsearch
es.update(index='hypothesis_v1',
                body={"doc": {"stanford": 1, "parsed_sents": parsed }})

NameError: name 'parsed' is not defined